In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
%matplotlib inline

# Data Loading

In [ ]:
ad_data = pd.read_csv('../input/advertising/advertising.csv')
ad_data.head(3)

# EDA

In [ ]:
ad_data.info()

In [ ]:
ad_data.describe()

In [ ]:
ad_data['Age'].hist(bins=30)
plt.xlabel('Age')

In [ ]:
ad_data.Timestamp=pd.to_datetime(ad_data.Timestamp)
ad_data['hour']=ad_data.Timestamp.apply(lambda x: x.hour)
ad_data['clicks_hour']=ad_data.groupby('hour')['Clicked on Ad'].transform('sum')
sns.lineplot(x='hour', y='clicks_hour', data=ad_data)

In [ ]:
ad_data['day']=ad_data.Timestamp.apply(lambda x: x.day_of_week)
ad_data['clicks_day']=ad_data.groupby('day')['Clicked on Ad'].transform('sum')
l=ad_data['day'].map({0: 'Sun', 1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat'})
sns.lineplot(l, y='clicks_day', data=ad_data)

In [ ]:
sns.scatterplot(x='Age', y='Area Income', hue='Clicked on Ad', data=ad_data)

In [ ]:
sns.scatterplot(x='Age', y='Daily Time Spent on Site', hue='Clicked on Ad', data=ad_data)

In [ ]:
sns.scatterplot(x='Daily Internet Usage', y='Daily Time Spent on Site', hue='Clicked on Ad', data=ad_data)

In [ ]:
sns.countplot(ad_data['Male'])

In [ ]:
sns.pairplot(ad_data,hue='Clicked on Ad')

# Data Preprocessing

In [ ]:
X = ad_data[['Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage', 'Male']]
y = ad_data['Clicked on Ad']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=58)

In [ ]:
scale=MinMaxScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

# Logistic Regression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
y_pred = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
lg=accuracy_score(y_test,y_pred)

# Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
rfs=accuracy_score(y_test,y_pred)

# XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
xgbs=accuracy_score(y_test,y_pred)

# SVM

In [ ]:
svm = SVC()
svm.fit(X_train,y_train)
y_pred = svm.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
svcs=accuracy_score(y_test,y_pred)

# Model comparison

In [ ]:
models = pd.DataFrame({
    'Model':['Logistic Regression','Random Forest', 'XGBoost', 'SVC'],
    'Accuracy_score' :[lg,rfs, xgbs, svcs]
})
sns.barplot(x='Accuracy_score', y='Model', data=models)
models.sort_values(by='Accuracy_score', ascending=False)